In [75]:
%pylab inline
from scipy.optimize import curve_fit
import pandas as pd
import pandas as pd
import seaborn as sns
import numpy as np
import scipy 
import matplotlib.pyplot as plt
import matplotlib.style as style
import itertools
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
import sys
import bookie_package as bp
import warnings
warnings.filterwarnings('ignore')


Populating the interactive namespace from numpy and matplotlib


In [35]:
df_both_seasons_away = pd.read_pickle('df_both_seasons_essentials')

## Add avg Away Team Goal Difference

In [36]:
df_both_seasons_away.head()

,Day,Month,Year,HomeTeam,AwayTeam,FTHG,FTAG,HS,AS,HST,...,HC,AC,HF,AF,HY,AY,HR,AR,HTGDIFF,ATGDIFF
139,18,12,2019,Leverkusen,Hertha,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140,18,12,2019,M'gladbach,Paderborn,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141,18,12,2019,Wolfsburg,Schalke 04,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
142,18,12,2019,Ein Frankfurt,FC Koln,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
143,18,12,2019,Freiburg,Bayern Munich,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
d_both_seasons = bp.averages.avg_goal_diff(df_both_seasons_away, 'AVGATGDIFF', 'AwayTeam', 'A')

In [38]:
df_both_seasons = bp.averages.from_dict_value_to_df(d_both_seasons)

In [39]:
df_both_seasons=df_both_seasons.sort_values(['Year', 'Month','Day'], ascending=False)

In [40]:
avg_ftag_per_team=bp.averages.avg_goals(df_both_seasons, 'AVGFTAG', 'AwayTeam', 'A')

In [41]:
df_both_seasons = bp.averages.from_dict_value_to_df(avg_ftag_per_team)

In [42]:
df_both_seasons=df_both_seasons.sort_values(['Year', 'Month','Day'], ascending=False)

In [43]:
df_both_seasons.head()

,Day,Month,Year,HomeTeam,AwayTeam,FTHG,FTAG,HS,AS,HST,...,HF,AF,HY,AY,HR,AR,HTGDIFF,ATGDIFF,AVGATGDIFF,AVGFTAG
139,18,12,2019,Leverkusen,Hertha,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.480000,1.320000
140,18,12,2019,M'gladbach,Paderborn,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.000000,1.142857
141,18,12,2019,Wolfsburg,Schalke 04,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.041667,1.458333
142,18,12,2019,Ein Frankfurt,FC Koln,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.625000,0.750000
143,18,12,2019,Freiburg,Bayern Munich,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,2.250000


## Add Columns with previous ATGDIFF for each AwayTeam

In [44]:
team_with_past_ATGDIFF=bp.averages.previous_data(df_both_seasons, 'AwayTeam', 'ATGDIFF')

In [45]:
df_team_with_past_ATGDIFF=bp.averages.from_dict_value_to_df(team_with_past_ATGDIFF)

In [46]:
df_team_with_past_ATGDIFF.head()

,AC,AF,AR,AS,AST,ATGDIFF,ATGDIFF_1,ATGDIFF_10,ATGDIFF_11,ATGDIFF_12,...,HC,HF,HR,HS,HST,HTGDIFF,HY,HomeTeam,Month,Year
139,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,-2.0,-5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Leverkusen,12,2019
117,1.0,15.0,0.0,9.0,3.0,0.0,-4.0,-2.0,-5.0,-1.0,...,16.0,11.0,0.0,25.0,6.0,0.0,3.0,Ein Frankfurt,12,2019
106,3.0,18.0,1.0,5.0,1.0,-4.0,-1.0,-5.0,-1.0,-1.0,...,3.0,8.0,0.0,14.0,5.0,4.0,0.0,Augsburg,11,2019
87,4.0,9.0,0.0,8.0,3.0,-1.0,0.0,-1.0,-1.0,3.0,...,1.0,14.0,0.0,16.0,6.0,1.0,1.0,Union Berlin,11,2019
68,5.0,11.0,0.0,11.0,2.0,0.0,4.0,-1.0,3.0,2.0,...,7.0,8.0,0.0,15.0,5.0,0.0,1.0,Werder Bremen,10,2019


In [47]:
df_team_with_past_ATGDIFF = df_team_with_past_ATGDIFF.reindex(columns=[
    'Day', 'Month', 'Year', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG',
    'HTGDIFF', 'ATGDIFF', 'AVGATGDIFF','AVGFTAG', 'HS', 'AS', 'HST', 'AST', 'HC', 'AC', 'HF', 'AF', 'HY', 'AY', 'HR', 'AR', 'ATGDIFF_1', 'ATGDIFF_2', 'ATGDIFF_3', 'ATGDIFF_4', 'ATGDIFF_5', 'ATGDIFF_6', 'ATGDIFF_7',
    'ATGDIFF_8', 'ATGDIFF_9', 'ATGDIFF_10', 'ATGDIFF_11', 'ATGDIFF_12', 'ATGDIFF_13', 'ATGDIFF_14', 'ATGDIFF_15', 'ATGDIFF_16', 'ATGDIFF_17', 'ATGDIFF_18', 'ATGDIFF_19',
    'ATGDIFF_20', 'ATGDIFF_21', 'ATGDIFF_22', 'ATGDIFF_23'
])

In [48]:
df_team_with_past_ATGDIFF.fillna(0, inplace=True)

In [49]:
team_with_past_AST=bp.averages.previous_data(df_team_with_past_ATGDIFF, 'AwayTeam', 'AST')

In [50]:
df_team_with_past_AST = bp.averages.from_dict_value_to_df(team_with_past_AST)

In [51]:
  df_team_with_past_AST = df_team_with_past_AST.reindex(columns=[ 'Day',
    'Month',
    'Year',
    'HomeTeam',
    'AwayTeam',
    'FTHG',
    'FTAG',
    'HTGDIFF',
    'ATGDIFF',
    'AVGATGDIFF',
    'AVGFTAG',
    'HS',
    'AS',
    'HST',
    'AST',
    'HC',
    'AC',
    'HF',
    'AF',
    'HY',
    'AY',
    'HR',
    'AR',
    'ATGDIFF_1',
    'ATGDIFF_2',
    'ATGDIFF_3',
    'ATGDIFF_4',
    'ATGDIFF_5',
    'ATGDIFF_6',
    'ATGDIFF_7',
    'ATGDIFF_8',
    'ATGDIFF_9',
    'ATGDIFF_10',
    'ATGDIFF_11',
    'ATGDIFF_12',
    'ATGDIFF_13',
    'ATGDIFF_14',
    'ATGDIFF_15',
    'ATGDIFF_16',
    'ATGDIFF_17',
    'ATGDIFF_18',
    'ATGDIFF_19',
    'ATGDIFF_20',
    'ATGDIFF_21',
    'ATGDIFF_22',
    'ATGDIFF_23',
    'AST_1',
    'AST_2',
    'AST_3',
    'AST_4',
    'AST_5',
    'AST_6',
    'AST_7',
    'AST_8',
    'AST_9',
    'AST_10',
    'AST_11',
    'AST_12',
    'AST_13',
    'AST_14',
    'AST_15',
    'AST_16',
    'AST_17',
    'AST_18',
    'AST_19',
    'AST_20',
    'AST_21',
    'AST_22',
    'AST_23',
    ])

In [52]:
team_with_past_FTAG = bp.averages.previous_data(df_team_with_past_AST, 'AwayTeam', 'FTAG')


In [53]:
df_team_with_past_FTAG = bp.averages.from_dict_value_to_df(team_with_past_FTAG)

In [54]:
df_team_with_past_FTAG = df_team_with_past_FTAG.reindex(columns=[ 'Day',
'Month',
'Year',
'HomeTeam',
'AwayTeam',
'FTHG',
'FTAG',
'HTGDIFF',
'ATGDIFF',
'AVGATGDIFF',
'AVGFTAG',
'HS',
'AS',
'HST',
'AST',
'HC',
'AC',
'HF',
'AF',
'HY',
'AY',
'HR',
'AR',
'ATGDIFF_1',
'ATGDIFF_2',
'ATGDIFF_3',
'ATGDIFF_4',
'ATGDIFF_5',
'ATGDIFF_6',
'ATGDIFF_7',
'ATGDIFF_8',
'ATGDIFF_9',
'ATGDIFF_10',
'ATGDIFF_11',
'ATGDIFF_12',
'ATGDIFF_13',
'ATGDIFF_14',
'ATGDIFF_15',
'ATGDIFF_16',
'ATGDIFF_17',
'ATGDIFF_18',
'ATGDIFF_19',
'ATGDIFF_20',
'ATGDIFF_21',
'ATGDIFF_22',
'ATGDIFF_23',
'AST_1',
'AST_2',
'AST_3',
'AST_4',
'AST_5',
'AST_6',
'AST_7',
'AST_8',
'AST_9',
'AST_10',
'AST_11',
'AST_12',
'AST_13',
'AST_14',
'AST_15',
'AST_16',
'AST_17',
'AST_18',
'AST_19',
'AST_20',
'AST_21',
'AST_22',
'AST_23',
'FTAG_1',
'FTAG_2',
'FTAG_3',
'FTAG_4',
'FTAG_5',
'FTAG_6',
'FTAG_7',
'FTAG_8',
'FTAG_9',
'FTAG_10',
'FTAG_11',
'FTAG_12',
'FTAG_13',
'FTAG_14',
'FTAG_15',
'FTAG_16',
'FTAG_17',
'FTAG_18',
'FTAG_19',
'FTAG_20',
'FTAG_21',
'FTAG_22',
'FTAG_23' ])

In [55]:
#a = []
#for i in range(1,n_awaygames):
 #   a.append('FTAG_{}'.format(i))

In [56]:
df_team_with_past_FTAG.sort_values(['Year', 'Month','Day'], ascending=False,inplace=True)

In [57]:
df_team_with_past_FTAG.fillna(0, inplace=True)

In [58]:
df_team_with_past_FTAG = df_team_with_past_FTAG[['Day', 'Month', 'Year', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG',
       'HTGDIFF', 'ATGDIFF', 'AVGATGDIFF', 'AVGFTAG', 'HS', 'AS', 'HST', 'AST',
       'HC', 'AC', 'HF', 'AF', 'HY', 'AY', 'HR', 'AR', 'ATGDIFF_1',
       'ATGDIFF_2', 'ATGDIFF_3', 'ATGDIFF_4', 'ATGDIFF_5', 'ATGDIFF_6',
       'ATGDIFF_7', 'ATGDIFF_8', 'ATGDIFF_9', 'ATGDIFF_10', 'AST_1', 'AST_2', 'AST_3', 'AST_4', 'AST_5',
       'AST_6', 'AST_7', 'AST_8', 'AST_9', 'AST_10', 'FTAG_1', 'FTAG_2', 'FTAG_3',
       'FTAG_4', 'FTAG_5', 'FTAG_6', 'FTAG_7', 'FTAG_8', 'FTAG_9', 'FTAG_10',
    ]]

In [59]:
df_team_with_past_FTAG.to_excel('df_team_with_past_FTAG_away.xlsx')

In [61]:
df_result = df_team_with_past_FTAG.copy()

In [62]:
df_result = df_result.drop(['HomeTeam', 'AwayTeam'], axis = 1)

In [63]:
print('Shape of features:', df_result.shape)

Shape of features: (450, 51)


## Features and Labels and Convert Data to Arrays

In [64]:
# Labels are the values we want to predict
labels = np.array(df_result['FTAG'])

# Remove the labels from the features
# axis 1 refers to the columns
df_result= df_result.drop(['Month','Day','Year','FTHG', 'FTAG', 'HTGDIFF', 'ATGDIFF', 'HS', 'AS', 'HST', 'AST', 'HC', 'AC', 'HF', 'AF', 'HY', 'AY', 'HR', 'AR'], axis = 1)

# Saving feature names for later use
feature_list = list(df_result.columns)

# Convert to numpy array
df_result = np.array(df_result)

## Training and Testing Sets

In [65]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(
    df_result, labels, test_size = 0.25,random_state = 42
)
                

In [66]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (337, 32)
Training Labels Shape: (337,)
Testing Features Shape: (113, 32)
Testing Labels Shape: (113,)


## Establish Baseline

In [67]:
# The baseline predictions are the historical averages
baseline_preds = train_features[:, feature_list.index('AVGFTAG')]
# AVerage goals made by home team

# Baseline errors, and display average baseline error
baseline_errors = abs(baseline_preds - train_labels)
print('Average baseline error: ', round(np.mean(baseline_errors), 2), 'Goals.')

Average baseline error:  0.99 Goals.


## Train Model

In [69]:
# Import the model we are using
#from sklearn.ensemble import RandomForestRegressor

# Instantiate model 
#rf = RandomForestRegressor(n_estimators= 1000, random_state=42)

# Train the model on training data
#rf.fit(train_features, train_labels);

In [68]:
rf = bp.prediction.random_forrest(train_features, train_labels, n_estimators=1000,random_state = 42)

## Make Predictions on Test Data

In [70]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)

# Calculate the absolute errors
errors = abs(predictions - test_labels)

# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'Goals.')

Mean Absolute Error: 0.98 Goals.


In [ ]:
for i in zip(np.round(predictions,0), test_labels):
    print(i)

## Determine Performance Metrics

In [71]:
# Calculate mean absolute percentage error (MAPE)
err = errors/test_labels
err[np.isnan(err)] = 0
mape = err * 100
mape[mape == inf] = 0

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 70.22 %.


## Interpret Model Results

### Visualizing a Single Decision Tree

In [72]:
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot
# Pull out one tree from the forest
tree = rf.estimators_[5]
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot
# Pull out one tree from the forest
tree = rf.estimators_[5]
# Export the image to a dot file
export_graphviz(tree, out_file = 'tree_away.dot', feature_names = feature_list, rounded = True, precision = 1)
# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree_away.dot')
# Write graph to a png file
graph.write_png('tree_away.png')

In [73]:
print('The depth of this tree is:', tree.tree_.max_depth)

The depth of this tree is: 20


In [76]:

# Limit depth of tree to 2 levels
rf_small = RandomForestRegressor(n_estimators=10, max_depth = 4, random_state=42)
rf_small.fit(train_features, train_labels)

# Extract the small tree
tree_small = rf_small.estimators_[5]

# Save the tree as a png image
export_graphviz(tree_small, out_file = 'small_tree_away.dot', feature_names = feature_list, rounded = True, precision = 1)

(graph, ) = pydot.graph_from_dot_file('small_tree_away.dot')

graph.write_png('small_tree_away.png');

### Variable Importances

In [77]:
importances = list(rf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 5)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: AVGATGDIFF           Importance: 0.13928
Variable: AVGFTAG              Importance: 0.08857
Variable: AST_1                Importance: 0.06815
Variable: AST_3                Importance: 0.0462
Variable: ATGDIFF_1            Importance: 0.04422
Variable: AST_7                Importance: 0.04215
Variable: ATGDIFF_5            Importance: 0.04019
Variable: AST_2                Importance: 0.03462
Variable: ATGDIFF_3            Importance: 0.03295
Variable: AST_4                Importance: 0.03227
Variable: ATGDIFF_4            Importance: 0.03168
Variable: AST_5                Importance: 0.03009
Variable: AST_6                Importance: 0.02866
Variable: ATGDIFF_2            Importance: 0.0272
Variable: AST_9                Importance: 0.02561
Variable: AST_10               Importance: 0.02386
Variable: ATGDIFF_8            Importance: 0.0236
Variable: ATGDIFF_6            Importance: 0.02176
Variable: FTAG_2               Importance: 0.02023
Variable: ATGDIFF_10           Imp

In [ ]:
# x_values = list(range(len(importances)))
# # Make a bar chart
# plt.bar(x_values, importances, orientation = 'vertical', color = 'r', edgecolor = 'k', linewidth = 1.2)
# # Tick labels for x axis
# plt.xticks(x_values, feature_list, rotation='vertical')
# # Axis labels and title
# plt.ylabel('Importance'); plt.xlabel('Variable'); plt.title('Variable Importances');

In [ ]:
# # List of features sorted from most to least important
# sorted_importances = [importance[1] for importance in feature_importances]
# sorted_features = [importance[0] for importance in feature_importances]
# # Cumulative importances
# cumulative_importances = np.cumsum(sorted_importances)
# # Make a line graph
# plt.plot(x_values, cumulative_importances, 'g-')
# # Draw line at 95% of importance retained
# plt.hlines(y = 0.95, xmin=0, xmax=len(sorted_importances), color = 'r', linestyles = 'dashed')
# # Format x ticks and labels
# plt.xticks(x_values, sorted_features, rotation = 'vertical')
# # Axis labels and title
# plt.xlabel('Variable'); plt.ylabel('Cumulative Importance'); plt.title('Cumulative Importances');

In [ ]:
# # Find number of features for cumulative importance of 95%
# # Add 1 because Python is zero-indexed
# print('Number of features for 95% importance:', np.where(cumulative_importances > 0.95)[0][0] + 1)

In [ ]:
# # Extract the names of the most important features
# important_feature_names = [feature[0] for feature in feature_importances[0:5]]
# # Find the columns of the most important features
# important_indices = [feature_list.index(feature) for feature in important_feature_names]
# # Create training and testing sets with only the important features
# important_train_features = train_features[:, important_indices]
# important_test_features = test_features[:, important_indices]
# # Sanity check on operations
# print('Important train features shape:', important_train_features.shape)
# print('Important test features shape:', important_test_features.shape)

In [ ]:
# rf_exp = RandomForestRegressor(n_estimators= 1000, random_state=100)
# rf_exp.fit(train_features, train_labels)

In [ ]:
# # Train the expanded model on only the important features
# rf_exp.fit(important_train_features, train_labels);
# # Make predictions on test data
# predictions = rf_exp.predict(important_test_features)
# # Performance metrics
# errors = abs(predictions - test_labels)
# print('Average absolute error:', round(np.mean(errors), 2), 'goals.')
# # Calculate mean absolute percentage error (MAPE)

# err = errors/test_labels
# err[np.isnan(err)] = 0
# mape = err * 100
# mape[mape == inf] = 0

# # Calculate and display accuracy
# accuracy = 100 - np.mean(mape)
# print('Accuracy:', round(accuracy, 2), '%.')

### Predictions and Actual Values

In [ ]:
# # Dates of training values
# months = df_result[:, feature_list.index('Month')]
# days = df_result[:, feature_list.index('Day')]
# years = df_result[:, feature_list.index('Year')]

# # List and then convert to datetime object
# dates = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) for year, month, day in zip(years, months, days)]
# dates = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in dates]

# # Dataframe with true values and dates
# true_data = pd.DataFrame(data = {'date': dates, 'actual': labels})

# # Dates of predictions
# months = test_features[:, feature_list.index('Month')]
# days = test_features[:, feature_list.index('Day')]
# years = test_features[:, feature_list.index('Year')]

# # Column of dates
# test_dates = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) for year, month, day in zip(years, months, days)]

# # Convert to datetime objects
# test_dates = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in test_dates]

# # Dataframe with predictions and dates
# predictions_data = pd.DataFrame(data = {'date': test_dates, 'prediction': predictions})

In [ ]:
# sns.set(style='whitegrid', rc={'figure.figsize':(11.7,8.27)})
# sns.set_context('talk')

In [ ]:
# # Plot the actual values
# plt.plot(true_data['date'], true_data['actual'], 'b-', label = 'actual')

# # Plot the predicted values
# plt.plot(predictions_data['date'], predictions_data['prediction'], 'ro', label = 'prediction')
# plt.xticks(rotation = '60'); 
# plt.legend()

# # Graph labels
# plt.xlabel('Date'); plt.ylabel('ATHG'); plt.title('Actual and Predicted Values');
